In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

import glob as gb
import cv2
import matplotlib.pylab as plt
plt.style.use('ggplot')

### Dataset source: https://www.kaggle.com/datasets/annielu21/house-rooms

In [2]:
import warnings

warnings.filterwarnings("ignore", message="All log messages before absl::InitializeLog() is called are written to STDERR")
warnings.filterwarnings("ignore", message="Fallback to op-by-op mode because memset node breaks graph update")

In [3]:
from sklearn.model_selection import train_test_split
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

2024-06-21 04:56:09.131752: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-21 04:56:09.131952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-21 04:56:09.304762: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [5]:
set_random_seed(42)

In [6]:
# Path gambar
img_path = '/kaggle/input/house-rooms/room-dataset'

input_dir = 'images'
output_dir = 'output'

# Membuat folder output jika belum ada
os.makedirs(output_dir, exist_ok=True)

In [7]:
# Membuat Folder Kategori
categories = ['bathroom', 'bedroom', 'dining', 'gaming', 
              'kitchen','laundry','living', 'office',
             'terrace', 'yard']

for category in categories:
    category_dir = os.path.join(output_dir, category)
    os.makedirs(category_dir, exist_ok=True)

In [8]:
# Memindahkan file ke folder yang dipilih
for root, dirs, files in os.walk(img_path):
    for file in files:
        category = os.path.basename(root)
        if category in categories:
            source_file = os.path.join(root, file)
            destination_dir = os.path.join(output_dir, category)
            shutil.copy(source_file, destination_dir)

# Menampilkan jumlah file dalam setiap kategori
for category in categories:
    category_dir = os.path.join(output_dir, category)
    num_files = len(os.listdir(category_dir))
    print(f"Total {category} files: {num_files}")

Total bathroom files: 740
Total bedroom files: 1083
Total dining files: 1178
Total gaming files: 1511
Total kitchen files: 1208
Total laundry files: 1589
Total living files: 1164
Total office files: 1241
Total terrace files: 1210
Total yard files: 1398


In [32]:
# Buat folder train dan test
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")
if os.path.exists(train_dir):
    shutil.rmtree(train_dir)
if os.path.exists(test_dir):
    shutil.rmtree(test_dir)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Split data menjadi 70%train 30%test
for category in categories:
    category_dir = os.path.join(output_dir, category)
    images = os.listdir(category_dir)
    train_images, test_images = train_test_split(images, test_size=0.3, random_state=42)
    
    # Pindahkan gambar-gambar ke dalam folder train dan test
    for img in train_images:
        target_dir = os.path.join(train_dir, category)
        os.makedirs(target_dir, exist_ok=True)
        shutil.move(os.path.join(category_dir, img), os.path.join(target_dir, img))

    for img in test_images:
        target_dir = os.path.join(test_dir, category)
        os.makedirs(target_dir, exist_ok=True)
        shutil.move(os.path.join(category_dir, img), os.path.join(target_dir, img))

In [33]:
# Cetak jumlah gambar dalam folder train
print("\nJumlah Gambar dalam Folder Train:")
for category in categories:
    train_category_dir = os.path.join(train_dir, category)
    num_train_images = len(os.listdir(train_category_dir))
    print(f"{category}: {num_train_images} gambar")

# Cetak jumlah gambar dalam folder test
print("\nJumlah Gambar dalam Folder Test:")
for category in categories:
    test_category_dir = os.path.join(test_dir, category)
    num_test_images = len(os.listdir(test_category_dir))
    print(f"{category}: {num_test_images} gambar")


Jumlah Gambar dalam Folder Train:
bathroom: 518 gambar
bedroom: 758 gambar
dining: 824 gambar
gaming: 1057 gambar
kitchen: 845 gambar
laundry: 1112 gambar
living: 814 gambar
office: 868 gambar
terrace: 847 gambar
yard: 978 gambar

Jumlah Gambar dalam Folder Test:
bathroom: 222 gambar
bedroom: 325 gambar
dining: 354 gambar
gaming: 454 gambar
kitchen: 363 gambar
laundry: 477 gambar
living: 350 gambar
office: 373 gambar
terrace: 363 gambar
yard: 420 gambar


In [ ]:
def convert_to_grayscale(img):
    # Konversi gambar ke grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return gray

In [ ]:
def apply_clahe(gray_img):
    # Terapkan CLAHE pada gambar grayscale
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(gray_img)
    
    # Gabungkan kembali ke tiga saluran RGB
    final_img = cv2.merge((clahe_img, clahe_img, clahe_img))
    return final_img

In [ ]:
from keras.preprocessing import image

def preprocess_with_clahe(img):
    # Konversi gambar ke array dengan tipe data uint8
    img_array = image.img_to_array(img, dtype='uint8')
    # Konversi gambar ke grayscale
    gray_img = convert_to_grayscale(img_array)
    # Terapkan CLAHE pada gambar grayscale
    img_clahe = apply_clahe(gray_img)
    # Lakukan preprocess_input VGG16
    img_preprocessed = preprocess_input(img_clahe)
    return img_preprocessed

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.applications.vgg16 import preprocess_input

def visualize_random_images(folder, num_images=1, figsize=(8, 4)):
    categories = os.listdir(folder)
    for category in categories:
        category_dir = os.path.join(folder, category)
        images = os.listdir(category_dir)
        random_images = random.sample(images, num_images)
        print(f"\nContoh Gambar untuk Kategori {category}:")
        for img_file in random_images:
            img_path = os.path.join(category_dir, img_file)
            img = mpimg.imread(img_path)
            img_clahe = preprocess_with_clahe(img)
            
            fig, axes = plt.subplots(1, 2, figsize=figsize)
            axes[0].imshow(img)
            axes[0].set_title('Original Image')
            axes[0].axis("off")
            
            axes[1].imshow(img_clahe)
            axes[1].set_title('CLAHE Image')
            axes[1].axis("off")
            
            plt.show()

print('Train Folder')
visualize_random_images(train_dir, figsize=(8, 4))
print('\nTest Folder')
visualize_random_images(test_dir, figsize=(8, 4))

In [ ]:
# np.random.seed(42)

# # Membuat objek ImageDataGenerator untuk pelatihan dengan preprocessing CLAHE
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     preprocessing_function=preprocess_with_clahe,
#     samplewise_center=True,
#     rotation_range=10,
#     zoom_range=0.1,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=True,
#     vertical_flip=False
# )

# traindata = train_datagen.flow_from_directory(
#     directory="./output/train", 
#     target_size=(224,224), 
#     color_mode="rgb", 
#     class_mode='categorical',
#     shuffle=False)

# # Membuat objek ImageDataGenerator untuk pengujian dengan preprocessing CLAHE
# test_datagen = ImageDataGenerator(
#     rescale=1./255,
#     preprocessing_function=preprocess_with_clahe,
#     samplewise_center=True
# )

# testdata = test_datagen.flow_from_directory(
#     directory="./output/test", 
#     target_size=(224,224), 
#     color_mode="rgb", 
#     class_mode='categorical',
#     shuffle=False)

In [ ]:
# Membuat objek ImageDataGenerator untuk pelatihan dengan preprocessing CLAHE
train_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_with_clahe,
    samplewise_center=True,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest'
)

traindata = train_datagen.flow_from_directory(
    directory="./output/train", 
    target_size=(224,224), 
    color_mode="rgb", 
    class_mode='categorical',
    shuffle=False)

# Membuat objek ImageDataGenerator untuk pengujian dengan preprocessing CLAHE
test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_with_clahe,
    samplewise_center=True
)

testdata = test_datagen.flow_from_directory(
    directory="./output/test", 
    target_size=(224,224), 
    color_mode="rgb", 
    class_mode='categorical',
    shuffle=False)

In [ ]:
testdata.class_indices

### MODELING

In [ ]:
# VGG16 model (VGG16 is a famous convolution neural net (CNN) architecture 
# invented by Karen Simonyan and Andrew Zisserman in 2014.)

from keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape=(224,224,3), include_top=False, weights='imagenet')

base_model.summary()

In [ ]:
# freeze semua layer kecuali 2,3,4 layer dari akhir
for layer in base_model.layers:
    layer.trainable = False
base_model.layers[-2].trainable = True
base_model.layers[-3].trainable = True
base_model.layers[-4].trainable = True

In [ ]:
# for layer in base_model.layers:
#     layer.trainable = False

In [ ]:
# Check if GPU is available
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Set memory growth for GPU
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from keras.regularizers import l2
import tensorflow as tf

# model = Sequential()
# model.add(Input(shape=(224, 224, 3)))
# model.add(base_model)
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(GlobalAveragePooling2D())

# # Adding additional Dense layers with Batch Normalization
# model.add(Dense(512, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# model.add(Dense(256, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# model.add(Dense(len(categories), activation='softmax'))

model = Sequential()
model.add(Input(shape=(224, 224, 3)))
model.add(base_model)
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.1))
model.add(Dense(len(categories), activation='softmax', kernel_regularizer=l2(0.01)))



# model = Sequential()
# model.add(Input(shape=(224, 224, 3)))
# model.add(base_model)
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dropout(0.2))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(len(categories), activation='softmax'))

model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD  # Import SGD optimizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

#maxpooling 2x2 86.30310893058777%
#maxpooling 3x3 85.60557961463928%
#avgpooling 2x2 85.54216623306274%
#avgpooling 3x3 82.81547427177429%

#maxpool 2x2 + grayscale clahe  87.38110065460205%

device_name = tf.test.gpu_device_name()

modelpath = './output/bestmodel.keras'

np.random.seed(42)

### train the model
with tf.device(device_name):
    
    optimizer = SGD(learning_rate=0.005)
    
    model.compile(
      optimizer=optimizer,
      loss='categorical_crossentropy',
      metrics=['accuracy'])

    # Callback EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Early stopping for val_accuracy
    early_stopping_acc = EarlyStopping(monitor='val_accuracy', patience=10, 
                                       restore_best_weights=True, mode='max')

    # Early stopping for val_loss
    early_stopping_loss = EarlyStopping(monitor='val_loss', patience=5, 
                                        restore_best_weights=True, mode='min')

    
    # Reduce learning rate when a metric has stopped improving
    reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, 
                                  patience=8, min_lr=0.000005, verbose=1)

    
    # Callback ModelCheckpoint untuk menyimpan bobot terbaik
    checkpoint = ModelCheckpoint(
        modelpath,
        monitor='val_accuracy',
        save_best_only=True,  # Hanya menyimpan bobot terbaik
        mode='max',  # Mode untuk membandingkan nilai metrik (misalnya 'max' untuk akurasi)
        verbose=1  # Menampilkan pesan saat menyimpan
    )

    # Train the model
    history = model.fit(x=traindata, validation_data=testdata, epochs=200, 
                        batch_size=32, callbacks=[early_stopping_acc, early_stopping_loss,
                                                  checkpoint,reduce_lr])

In [ ]:
#Test the model with the test set

model.evaluate(testdata)

In [ ]:
# Visualize the model

plt.plot(history.history['accuracy'], marker='o')
plt.plot(history.history['val_accuracy'], marker='o')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()

In [ ]:
plt.plot(history.history['loss'], marker='o')
plt.plot(history.history['val_loss'], marker='o')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Prediksi label kelas menggunakan model terhadap data uji
predictions = model.predict(testdata)
predicted_classes = np.argmax(predictions, axis=1)

# Ambil label sebenarnya dari data uji
true_classes = testdata.labels

# Dapatkan nama kelas
class_labels = list(testdata.class_indices.keys())

# Buat classification matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Print classification matrix
print("Classification Matrix:")
print(cm)

# Print classification report
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

Accuracy adalah rasio dari jumlah prediksi yang benar (true predictions) dibagi dengan jumlah total instance dalam dataset. 

Precision mengukur seberapa akurat model dalam memprediksi kelas positif. Ini dihitung sebagai rasio dari true positive (TP) dibagi oleh jumlah true positive dan false positive (FP). 

 Recall mengukur seberapa baik model dalam menemukan semua instance yang benar dari kelas yang diinginkan. Ini dihitung sebagai rasio dari true positive (TP) dibagi oleh jumlah true positive dan false negative (FN). 

F1-score adalah rata-rata harmonik dari precision dan recall. Ini memberikan keseimbangan antara precision dan recall.

### Mencoba dengan dataset lain

### Dataset source: https://www.kaggle.com/datasets/annielu21/house-rooms

In [ ]:
import os
import shutil

# Menggunakan direktori yang dapat ditulis
img_path2 = '/kaggle/input/house-rooms/room-dataset'
input_dir = 'images'
output_dir2 = 'new_output'

# Membuat folder output jika belum ada
os.makedirs(output_dir2, exist_ok=True)

# Membuat Folder Kategori
categories = ['bath', 'bed', 'din', 'kitchen', 'living']
for category in categories:
    category_dir = os.path.join(output_dir2, category)
    os.makedirs(category_dir, exist_ok=True)

In [ ]:
folder_mapping = {
    'bathroom': 'bath',
    'bedroom': 'bed',
    'dining': 'din',
    'kitchen': 'kitchen',
    'living': 'living'
}

In [ ]:
# Memindahkan file ke folder yang dipilih
for root, dirs, files in os.walk(img_path2):
    for file in files:
        category = os.path.basename(root)
        if category in folder_mapping:
            source_file = os.path.join(root, file)
            destination_dir = os.path.join(output_dir2, folder_mapping[category])
            shutil.copy(source_file, destination_dir)

In [ ]:
for category in categories:
    category_dir = os.path.join(output_dir2, category)
    num_files = len(os.listdir(category_dir))
    print(f"Total {category} files: {num_files}")

In [ ]:
# Define the ImageDataGenerator for the new dataset
new_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_input
)

# Create a generator for the new dataset
new_generator = new_datagen.flow_from_directory(
    directory=output_dir2,
    classes=['bath', 'bed', 'din', 'kitchen', 'living'],
    target_size=(224,224), 
    color_mode="rgb", 
    class_mode='categorical',
    batch_size=32,  # Specify the batch size
    shuffle=False  # Do not shuffle the data for testing
)

In [ ]:
def create_model():

    np.random.seed(42)

    model = Sequential()
    model.add(Input(shape=(224,224,3)))
    model.add(base_model)
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(len(categories), activation='softmax'))
    
    
    optimizer = SGD(learning_rate=0.005)
    
    model.compile(
      optimizer=optimizer,
      loss='categorical_crossentropy',
      metrics=['accuracy'])
    
    return model

model2 = create_model()

In [ ]:
model2.summary()

In [ ]:
model2.load_weights('/kaggle/working/output/bestmodel.keras')

In [ ]:
model2.evaluate(new_generator)

### Didapatkan akurasi 80% untuk dataset yang lain